# Multinormial Logistic regression
## softmax Classifier



### 필요한 모듈 세팅

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.1.0


### 데이터 세팅

In [120]:
#데이터를 세팅했습니다.
#x데이터는 4개씩 8슬라이스로 나뉜 데이터를 준비했습니다.
x_data = [[1, 2, 4, 1],
          [2, 2, 3, 2],
          [3, 7, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 1, 5],
          [1, 2, 5, 6],
          [2, 5, 6, 7],
          [1, 7, 7, 7]]
#y데이터는 3개씩 8개 슬라이스로 나뉜 데이터를 준비했구요,
y_data = [[0, 0, 1],
          [0, 0, 1],
          [1, 0, 0],
          [0, 1, 0],
          [0, 1, 0],
          [0, 0, 1],
          [1, 0, 0],
          [1, 0, 0]]
#즉 [1,2,4,1] =[0,0,1] 처럼 대치되는 것입니다.

In [121]:
x_data

[[1, 2, 4, 1],
 [2, 2, 3, 2],
 [3, 7, 3, 4],
 [4, 1, 5, 5],
 [1, 7, 1, 5],
 [1, 2, 5, 6],
 [2, 5, 6, 7],
 [1, 7, 7, 7]]

In [122]:
#텐서플로우 함수를 이용하기전에 데이터를 넘피 배열로 변환 해줍니다
x_data = np.array(x_data, dtype=np.float32)
x_data

array([[1., 2., 4., 1.],
       [2., 2., 3., 2.],
       [3., 7., 3., 4.],
       [4., 1., 5., 5.],
       [1., 7., 1., 5.],
       [1., 2., 5., 6.],
       [2., 5., 6., 7.],
       [1., 7., 7., 7.]], dtype=float32)

In [123]:
#텐서플로우 함수를 이용하기전에 데이터를 넘피 배열로 변환 해줍니다
y_data = np.array(y_data, dtype =np.float32)
y_data

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

### dataset준비

In [124]:
#학습시클 값을 dataset에 담아준다.
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
#repeat()은 반복, batch()는 묶음을 의미한다.
#반복을 통해 학습할 데이터의 양을 늘려준다.
#묶음을 통해 학습속도?를 조절 한다.
dataset = dataset.repeat(2).batch(2)
#dataset = dataset.batch(2)
dataset

<BatchDataset shapes: ((None, 4), (None, 3)), types: (tf.float32, tf.float32)>

In [125]:
#dataset값 확인하기
#repeat과 batch에 따라 len(elem)이 달라지는것을 확인 가능하다.
elem = [i for i in dataset]
print(elem)
print(len(elem))

[(<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[1., 2., 4., 1.],
       [2., 2., 3., 2.]], dtype=float32)>, <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 1.]], dtype=float32)>), (<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[3., 7., 3., 4.],
       [4., 1., 5., 5.]], dtype=float32)>, <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.]], dtype=float32)>), (<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[1., 7., 1., 5.],
       [1., 2., 5., 6.]], dtype=float32)>, <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 0., 1.]], dtype=float32)>), (<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[2., 5., 6., 7.],
       [1., 7., 7., 7.]], dtype=float32)>, <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [1., 0., 0.]], dtype=float32)>), (<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[1., 2., 4., 1.],
       [2., 2., 3., 2

### 분류 갯수 선언
현재 y값을 [0,0,1]과같이 3개분류로 나누었기 때문에

In [126]:
#분류가짓수를 3으로 선언
nb_classes = 3

In [127]:
#즉 y값의 열과 같은값입니다.
print(y_data.shape[1])

3


### W,b 할당해 주기
softmax를 사용을 위해, W값을 배열((x값의 열값),(y값의 열값)으로 마춰줘야한다.   
b값은 ((y값의 열값),)으로 맞춰준다

In [128]:
W = tf.Variable(tf.random.normal((4, nb_classes)), name = 'weight')
b = tf.Variable(tf.random.normal((nb_classes,)),name = 'bias')

In [129]:
W

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.09651633, -0.69214475, -0.49522713],
       [ 0.15072225,  2.0019443 , -0.70925987],
       [-0.7252303 ,  0.23252739, -2.033938  ],
       [ 0.21821043,  0.5606956 ,  1.0457467 ]], dtype=float32)>

In [130]:
b

<tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 0.5649487 , -0.44032323,  0.35004106], dtype=float32)>

In [131]:
#그리고 변수값에 담아준다.
variables = [W,b]
variables

[<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 0.09651633, -0.69214475, -0.49522713],
        [ 0.15072225,  2.0019443 , -0.70925987],
        [-0.7252303 ,  0.23252739, -2.033938  ],
        [ 0.21821043,  0.5606956 ,  1.0457467 ]], dtype=float32)>,
 <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 0.5649487 , -0.44032323,  0.35004106], dtype=float32)>]

### softmax함수
변수를 sotfmax함수에 적용시킨후 값을 리턴 받는다.

In [132]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X,W)+b)

### softmax 테스트

In [133]:
testarr = [[5,6,1,3],[1,2,3,4],[4,3,2,1],[1,2,3,4]]
testarr = np.array(testarr, dtype = np.float32)

print(hypothesis(testarr))

tf.Tensor(
[[2.9039767e-04 9.9970931e-01 2.2574139e-07]
 [2.1250849e-03 9.9778324e-01 9.1741429e-05]
 [2.5288910e-02 9.7468692e-01 2.4201512e-05]
 [2.1250849e-03 9.9778324e-01 9.1741429e-05]], shape=(4, 3), dtype=float32)


### cost함수
cost(에러율,오차)을 찾는 함수

In [134]:
def cost_fn(X,Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis =1 )
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(10.610207, shape=(), dtype=float32)


### 경사하강법 함수

In [137]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.00101466, -0.00313479,  0.0021202 ],
       [-0.00215435,  0.00115918,  0.00099536],
       [-0.00142222,  0.00508382, -0.00366149],
       [ 0.00123679, -0.0037799 ,  0.00254326]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.00488178, -0.00058013, -0.00430162], dtype=float32)>]


### 반복 및 결과 값 도출

In [138]:
def fit(X, Y, epochs=2000, verbose=1000):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 0.026241
Loss at epoch 1000: 0.017634
Loss at epoch 2000: 0.013287


In [142]:
W

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-1.2719332 ,  2.5360806 , -2.3549967 ],
       [ 2.3983405 , -0.47378036, -0.4811593 ],
       [ 0.41838354, -5.3999457 ,  2.4549253 ],
       [-0.56878024,  4.0097694 , -1.6163393 ]], dtype=float32)>

In [40]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[0.00419643 0.00761949 0.9881841 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [41]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[1.3917855e-03 1.8417474e-09 9.9860829e-01]
 [4.5423996e-02 6.9806455e-03 9.4759536e-01]
 [9.7841245e-01 2.1580271e-02 7.2500425e-06]
 [3.7434872e-04 9.7464120e-01 2.4984468e-02]
 [2.5994429e-02 9.7400552e-01 2.1482348e-08]
 [1.8975602e-02 1.7405065e-02 9.6361935e-01]
 [9.6871400e-01 3.2455898e-03 2.8040400e-02]
 [9.9671632e-01 8.1378770e-09 3.2836695e-03]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 0 1 1 2 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 0 1 1 2 0 0], shape=(8,), dtype=int64)


In [2]:
xy = np.loadtxt('data-04-zoo.csv', delimiter = ',', dtype=np.float32)

x_data = xy[:, 0:-1]
y_data = xy[:, -1]

nb_classes = 7

Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


In [27]:
print(xy.shape)
xy

(101, 17)


array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 3.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 6.],
       [0., 1., 1., ..., 0., 0., 1.]], dtype=float32)

In [28]:
x_data

array([[1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 1.],
       [0., 0., 1., ..., 1., 0., 0.],
       ...,
       [1., 0., 0., ..., 1., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 1., 0., 0.]], dtype=float32)

In [29]:
y_data

array([0., 0., 3., 0., 0., 0., 0., 3., 3., 0., 0., 1., 3., 6., 6., 6., 1.,
       0., 3., 0., 1., 1., 0., 1., 5., 4., 4., 0., 0., 0., 5., 0., 0., 1.,
       3., 0., 0., 1., 3., 5., 5., 1., 5., 1., 0., 0., 6., 0., 0., 0., 0.,
       5., 4., 6., 0., 0., 1., 1., 1., 1., 3., 3., 2., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 6., 3., 0., 0., 2., 6., 1., 1., 2., 6., 3., 1., 0.,
       6., 3., 1., 5., 4., 2., 2., 3., 0., 0., 1., 0., 5., 0., 6., 1.],
      dtype=float32)

In [30]:
Y_one_hot

<tf.Tensor: shape=(101, 7), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1

In [39]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def logit_fn(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, 
                                                      from_logits=True)    
    cost = tf.reduce_mean(cost_i)    
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
    
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

In [40]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
#             print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy() 
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

Steps: 1 Loss: 3.799741268157959, Acc: 0.2772277295589447
Steps: 100 Loss: 0.5048637986183167, Acc: 0.8415841460227966
Steps: 200 Loss: 0.30533459782600403, Acc: 0.9306930899620056
Steps: 300 Loss: 0.22628118097782135, Acc: 0.9603960514068604
Steps: 400 Loss: 0.18192091584205627, Acc: 0.9603960514068604
Steps: 500 Loss: 0.1528809368610382, Acc: 0.9801980257034302
Steps: 600 Loss: 0.1321880966424942, Acc: 0.9900990128517151
Steps: 700 Loss: 0.11662240326404572, Acc: 0.9900990128517151
Steps: 800 Loss: 0.10445830971002579, Acc: 0.9900990128517151
Steps: 900 Loss: 0.09467662870883942, Acc: 1.0
Steps: 1000 Loss: 0.08663266897201538, Acc: 1.0
